In [47]:
import numpy as np
import inspect

import likelihood

def my_model(K1,K2=10,non_float="test"):
    
    return K2*K2

m = likelihood.ModelWrapper(my_model)

In [58]:
m.__qualname__

AttributeError: 'ModelWrapper' object has no attribute '__qualname__'

In [57]:
isinstance(m,likelihood.model_wrapper.ModelWrapper)

True

In [45]:
a = m.model
a.__self__.param_names

yo


array([], dtype=float64)

In [25]:
a

AttributeError: 'function' object has no attribute 'self'

In [21]:
def yo():
    return None

yo.__qualname__

'yo'

In [ ]:


class ModelWrapper:
    """
    Wraps a model.  
    """
    
    #_mw_ pre-prepended to internal variables to avoid contaminating 
    # class namespace when we start sticking arbitrary model arguments
    # into self.__dict__
    
    # Has to be defined across class because we are going to hijack
    # __setattr__ and need to look inside this as soon as we start
    # setting attributes. 
    _mw_fit_parameters = {}
    
    def __init__(self,model_to_fit):
                
        self._model_to_fit = model_to_fit
        self._mw_load_model()
    
    def _mw_load_model(self):

        self._mw_fit_parameters = {}
        self._mw_other_arguments = {}
        
        getting_fit_params = True
        self._mw_signature = inspect.signature(self._model_to_fit)
        for p in self._mw_signature.parameters:
  
            # Make sure that this parameter name isn't already being used by
            # the wrapper. 
            try:
                self.__dict__[p]
                raise ValueError
            except ValueError:
                err = f"Parameter name '{p}' reserved by this class.\n"
                err += "Please change the argument name in your function.\n"
                raise ValueError(err)
            except KeyError:
                pass

            # If we hit args or kwargs, stop looking for fittable parameters
            if p in ['args','kwargs']:
                getting_fit_params = False
            
            # Try to turn the argument default into a guess.  If this fails b/c 
            # default is not coercable into a float, stop trying to grab fit 
            # parameters. 
            if getting_fit_params:
                if self._mw_signature.parameters[p].default == inspect._empty:
                    guess = None
                else:
                    guess = self._mw_signature.parameters[p].default
                    try:
                        guess = np.float(guess)
                    except (TypeError,ValueError):
                        getting_fit_params = False
            
            # If we are still getting fit parameters, record this as a fit parameter
            if getting_fit_params:
                self._mw_fit_parameters[p] = likelihood.FitParameter(name=p,guess=guess)
                self.__dict__[p] = self._mw_fit_parameters[p]
                
            # Otherwise, this is a standard model argument
            else:
                self._mw_other_arguments[p] = self._mw_signature.parameters[p].default
                self.__dict__[p] = self._mw_other_arguments[p]

                
    def __setattr__(self, key, value):
        """
        Hijack __setattr__ so setting the value for fit parameters
        updates the fit guess. 
        """
        
        # If we are not setting a fit parameter, just set it like normal
        if key not in self._mw_fit_parameters.keys():
            super(ModelWrapper, self).__setattr__(key, value)
            return
        
        # Otherwise, we're setting the guess of the fit parameter
        self._mw_fit_parameters[key].guess = value
    
    
    def _mw_update_observable(self):
        """
        """
        
        self._param_to_p_map = []
        self._mw_kwargs = {}
        counter = 0
        for p in self._mw_fit_parameters.keys():
            if self._mw_fit_parameters[p].fixed:
                self._mw_kwargs[p] = self._mw_fit_parameters[p].value
            else:
                self._mw_kwargs[p] = None
                counter += 1
        
        self._mw_kwargs.update(self._mw_other_parameters)

                
    def mw_observable(self,params):
    
        for i in range(len(params)):
            self._mw_kwargs[self._param_to_p_map[i]] = params[i]
        
        return self._model_to_fit(**self._mw_kwargs)
        
             
    @property
    def fit_parameters(self):
        """
        """
        
        return self._mw_fit_parameters

    @property
    def other_arguments(self):
        """
        """
        
        return self._mw_other_arguments
    
                
mw = ModelWrapper(my_model)

for p in mw.fit_parameters.keys():
    mw.fit_parameters[p].bounds = [0,10]
    
mw.fit_parameters["K2"].guess = 6
mw.K2.guess = 6


In [ ]:
from likelihood import ModelWrapper, MLFitter

mw = ModelWrapper(my_model)
mf = MLFitter(mw)
mf.fit()


In [ ]:
real = 0.1
num_steps = 50
scale_factor = 2

rtol = 0.1
atol = np.abs(real)*0.2

a_real = np.abs(real)
step_size = a_real/(num_steps//4)
bottom = -(a_real*scale_factor)
top = a_real*scale_factor + step_size

for x in np.arange(bottom,top,step_size):
    print(real,x,np.isclose(real,x,rtol=rtol,atol=atol))


In [ ]:
import inspect
def f(yo,junk):
    print(f)
    
inspect.isfunction(f)
len(inspect.signature(f).parameters)

In [ ]:
isinstance(np.array([1,2,3]),np.ndarray)

In [ ]:
isinstance(np.array([1,2,3]).dtype,np.int)

In [ ]:
issubclass(n.dtype('int8').type, n.integer)

In [ ]:
np.array(1,dtype=np.float)

In [ ]:
try:
    np.array(np.array([1,2,"a"],dtype=np.str),dtype=np.float)
except ValueError as err:
    err = f"{err}\n\nguesses must be a list or array of floats\n\n"
    raise ValueError(err)

In [ ]:
class Yo:
    
    def __init__(self):
        
        self.peach = 5
    
    def yo(self):
        
        self.__dict__["peach"]
        self.__dict__["test"]
        
        
x = Yo() 
        
        

In [ ]:
x.yo()